In [77]:
import pandas as pd
import numpy as np
import math

In [107]:
df=pd.read_csv('result.txt')
df.head(5)

,Trial,ChartType,RealRatio,InputRatio,indexDif
0,1,Pie,0.555556,0.50,6
1,2,Pie,0.333333,0.30,2
2,3,Bubble,0.433770,0.50,0
3,4,Pie,0.700000,0.60,1
4,5,Bubble,0.895625,0.99,0


In [108]:
#compute error
error=abs(df.RealRatio-df.InputRatio)*100+(1/8)
error[error<1]=1
df['log']=pd.DataFrame(np.log2(error))
df.head(5)

,Trial,ChartType,RealRatio,InputRatio,indexDif,log
0,1,Pie,0.555556,0.50,6,2.506032
1,2,Pie,0.333333,0.30,2,1.790077
2,3,Bubble,0.433770,0.50,0,2.754466
3,4,Pie,0.700000,0.60,1,3.339850
4,5,Bubble,0.895625,0.99,0,3.257394


In [109]:
#Pie chart analysis
Piedf=df[df.ChartType=="Pie"].reset_index(drop=True)
Piedf.head(5)

,Trial,ChartType,RealRatio,InputRatio,indexDif,log
0,1,Pie,0.555556,0.5,6,2.506032
1,2,Pie,0.333333,0.3,2,1.790077
2,4,Pie,0.700000,0.6,1,3.339850
3,6,Pie,0.545455,0.4,1,3.874842
4,8,Pie,0.250000,0.2,2,2.357552
